In [14]:
import time

In [15]:
def encode_varint(x):
    ret = b''
    while True:
        ret += bytes([x & 0x7f])
        if x < 0x80:
            break
        x >>= 7
    return ret

def decode_varint(x):
    ret = 0
    for i in range(len(x)):
        ret += x[i] << (7 * i)
        if x[i] < 0x80:
            return ret

In [16]:
def put_varint(x):
    global buf
    buf += encode_varint(x)

In [17]:
def put_u8(x):
    global buf
    buf += x.to_bytes(1, "big")

def put_u64(x):
    global buf
    buf += x.to_bytes(8, 'big')

In [18]:
def put_bytes(x):
    global buf
    put_varint(len(x))
    buf += x

def put_string(x):
    global buf
    buf += x.encode("utf-8")
    buf += b'\0'

In [19]:
def put_time():
    global buf
    put_u64(int(time.time()))

In [20]:
def file(name, data):
    global buf
    put_u8(0x00)
    put_string(name)
    put_time()
    put_bytes(data)

In [21]:
class Folder:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        global buf
        put_u8(0x01)
        put_string(self.name)
        put_time()

    
    def __exit__(self, type, value, traceback):
        global buf
        put_u8(0xFF)

In [22]:
buf = b'vmkp'

with Folder("/"):
    file("test.txt", b"test")
    file("meta.json", b'{"name": "test"}')
    
    with Folder("scenes"):
        file("meta.json", b'{"cnt": 200}')


In [23]:
buf

b'vmkp\x01/\x00\x00\x00\x00\x00b%fx\x00test.txt\x00\x00\x00\x00\x00b%fx\x04test\x00meta.json\x00\x00\x00\x00\x00b%fx\x10{"name": "test"}\x01scenes\x00\x00\x00\x00\x00b%fx\x00meta.json\x00\x00\x00\x00\x00b%fx\x0c{"cnt": 200}\xff\xff'

In [24]:
with open("test.vmkp", "wb") as fp:
    fp.write(buf)

In [25]:
[x for x in buf]

[118,
 109,
 107,
 112,
 1,
 47,
 0,
 0,
 0,
 0,
 0,
 98,
 37,
 102,
 120,
 0,
 116,
 101,
 115,
 116,
 46,
 116,
 120,
 116,
 0,
 0,
 0,
 0,
 0,
 98,
 37,
 102,
 120,
 4,
 116,
 101,
 115,
 116,
 0,
 109,
 101,
 116,
 97,
 46,
 106,
 115,
 111,
 110,
 0,
 0,
 0,
 0,
 0,
 98,
 37,
 102,
 120,
 16,
 123,
 34,
 110,
 97,
 109,
 101,
 34,
 58,
 32,
 34,
 116,
 101,
 115,
 116,
 34,
 125,
 1,
 115,
 99,
 101,
 110,
 101,
 115,
 0,
 0,
 0,
 0,
 0,
 98,
 37,
 102,
 120,
 0,
 109,
 101,
 116,
 97,
 46,
 106,
 115,
 111,
 110,
 0,
 0,
 0,
 0,
 0,
 98,
 37,
 102,
 120,
 12,
 123,
 34,
 99,
 110,
 116,
 34,
 58,
 32,
 50,
 48,
 48,
 125,
 255,
 255]